In [181]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
from pathlib import Path
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [182]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"
PATH = Path(path)
TRN_CSV = pd.read_csv(PATH/'train.csv')



In [180]:
TRN_CSV_COPY = TRN_CSV.copy()

In [183]:
cat_vars = ['user_id', 'region', 'city', 'parent_category_name', 'category_name', 
            'param_1', 'param_2', 'param_3', 'title', 'description', 'user_type']
cont_vars = ['price', 'item_seq_number', 'image_top_1']
drop_vars = ['']

In [184]:
[c for c in cat_vars ]

['user_id',
 'region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'title',
 'description',
 'user_type']

In [185]:
[c for c in cont_vars]

['price', 'item_seq_number', 'image_top_1']

In [186]:
for c in cat_vars:
    TRN_CSV_COPY[c] = TRN_CSV_COPY[c].astype("category").cat.as_ordered()


In [187]:
for c in cont_vars:
    TRN_CSV_COPY[c] = TRN_CSV_COPY[c].astype("float32")

In [188]:
TRN_CSV_COPY.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2.0,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19.0,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9.0,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286.0,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3.0,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [190]:
TRN_CSV_COPY.to_feather('trn_csv_copy_feather')

In [192]:
cat_vars

['user_id',
 'region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'title',
 'description',
 'user_type']

In [194]:
cat_sz = [(c, len(TRN_CSV_COPY[c].cat.categories)+1) for c in cat_vars]

In [196]:
emb_sz = [(c, min(50, (c+1)//2)) for _, c in cat_sz]

In [197]:
emb_sz

[(771770, 50),
 (29, 15),
 (1734, 50),
 (10, 5),
 (48, 24),
 (372, 50),
 (272, 50),
 (1220, 50),
 (788378, 50),
 (1317103, 50),
 (4, 2)]

In [200]:
embs = nn.ModuleList([nn.Embedding(c, s) for c, s in emb_sz])

In [202]:
x = embs[0]

In [204]:
x = x.weight.data

In [205]:
sc = 2/(x.size(1)+1)

In [206]:
x.uniform_(-sc, sc)


1.00000e-02 *
-1.1313  1.6932 -0.6664  ...  -3.5907 -2.2907  2.4591
 3.0368 -2.7596  3.7110  ...  -1.7509  1.0821 -2.8698
 2.2238 -2.4207 -1.7430  ...  -1.7231  1.0884 -2.0064
          ...             ⋱             ...          
 0.3840  1.5335 -3.7561  ...   2.6357 -2.7372 -2.5226
-1.9757 -3.4379  1.0886  ...   2.9428 -0.0122  3.6865
-2.6442 -2.6673 -1.1903  ...  -3.4608  3.6778  2.2395
[torch.FloatTensor of size 771770x50]

In [195]:
cat_sz

[('user_id', 771770),
 ('region', 29),
 ('city', 1734),
 ('parent_category_name', 10),
 ('category_name', 48),
 ('param_1', 372),
 ('param_2', 272),
 ('param_3', 1220),
 ('title', 788378),
 ('description', 1317103),
 ('user_type', 4)]

In [208]:
cat_sz

[('user_id', 771770),
 ('region', 29),
 ('city', 1734),
 ('parent_category_name', 10),
 ('category_name', 48),
 ('param_1', 372),
 ('param_2', 272),
 ('param_3', 1220),
 ('title', 788378),
 ('description', 1317103),
 ('user_type', 4)]

In [210]:
emb_sz

[(771770, 50),
 (29, 15),
 (1734, 50),
 (10, 5),
 (48, 24),
 (372, 50),
 (272, 50),
 (1220, 50),
 (788378, 50),
 (1317103, 50),
 (4, 2)]

In [211]:
for c in cat_vars:
    TRN_CSV_COPY[c] = TRN_CSV_COPY[c].cat.codes

In [209]:
TRN_CSV_COPY

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2.0,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19.0,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9.0,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286.0,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3.0,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9.0,2017-03-28,Private,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796.0,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125.0,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61.0,2017-03-25,Private,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567.0,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85.0,2017-03-17,Private,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415.0,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136.0,2017-03-22,Company,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46.0,0.00000


In [212]:
TRN_CSV_COPY = TRN_CSV_COPY.fillna(0)

In [214]:
sample = TRN_CSV_COPY[:1000]

In [216]:
        df_cat = sample[cat_vars]
        cat_cols = [c.values for n, c in df_cat.items()]
        cats = np.stack(cat_cols, 1).astype(np.int64)

In [217]:
cats

array([[ 675853,      19,     460,       4,      42,     248,      -1,      -1,  298998,  301412,       1],
       [ 173962,      17,    1300,       2,      22,     121,      -1,      -1,  676606, 1078274,       1],
       [ 440069,      16,    1276,       0,       2,      83,      -1,      -1,   79843,  140789,       1],
       [ 576929,      21,     940,       4,      42,      37,      -1,      -1,  109705,  759896,       0],
       [ 721853,       4,     317,       6,       0,     277,     118,      44,  161476,  172372,       1],
       [ 566683,      21,    1657,       4,      42,      37,      -1,      -1,  109147,  138234,       1],
       [  27020,      11,     975,       2,      38,     282,      -1,      -1,  183108, 1179892,       1],
       [ 768696,      15,    1153,       4,      29,     125,     135,      71,  137761,   90120,       1],
       [  16061,      14,    1107,       4,      29,     125,     202,     212,  507235,   27988,       1],
       [ 753379,      11,   

In [223]:
len(), len(emb_sz)

(11, 11)

In [224]:
len(embs)

11

In [274]:
x_cat = Variable(torch.from_numpy(cats[:1]))

In [276]:
x_cat

Variable containing:

Columns 0 to 5 
 6.7585e+05  1.9000e+01  4.6000e+02  4.0000e+00  4.2000e+01  2.4800e+02

Columns 6 to 10 
-1.0000e+00 -1.0000e+00  2.9900e+05  3.0141e+05  1.0000e+00
[torch.LongTensor of size 1x11]

In [277]:
[embs[i](x_cat[:,i]) for i, e in enumerate(embs)]

RuntimeError: index out of range at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/TH/generic/THTensorMath.c:277

In [261]:
len()

2

In [264]:
[e for i, e in enumerate(embs)]

[Embedding(771770, 50),
 Embedding(29, 15),
 Embedding(1734, 50),
 Embedding(10, 5),
 Embedding(48, 24),
 Embedding(372, 50),
 Embedding(272, 50),
 Embedding(1220, 50),
 Embedding(788378, 50),
 Embedding(1317103, 50),
 Embedding(4, 2)]

In [279]:
[e for i, e in enumerate(embs)]

[Embedding(771770, 50),
 Embedding(29, 15),
 Embedding(1734, 50),
 Embedding(10, 5),
 Embedding(48, 24),
 Embedding(372, 50),
 Embedding(272, 50),
 Embedding(1220, 50),
 Embedding(788378, 50),
 Embedding(1317103, 50),
 Embedding(4, 2)]

In [295]:
e = embs[11]

IndexError: index 11 is out of range

In [314]:
x_cat[:,0].data[0]

675853

In [321]:
embs[0](Variable(torch.LongTensor([0])))

Variable containing:

Columns 0 to 9 
1.00000e-02 *
 -1.1313  1.6932 -0.6664 -2.3058  3.3835  3.0775 -2.3301  0.9602  1.3240 -3.1918

Columns 10 to 19 
1.00000e-02 *
  3.5261  3.3214  1.6468 -2.3949 -1.0301 -0.7577 -0.1887  0.9411 -1.8591  2.8922

Columns 20 to 29 
1.00000e-02 *
 -3.4997 -1.2488  0.8480  2.5438 -3.5589  1.3226 -1.8172 -3.0742 -0.6431 -0.4052

Columns 30 to 39 
1.00000e-02 *
 -0.7498  0.4902  1.0694 -1.6423  1.3567 -0.7583  0.4738  0.8188  2.2279 -0.7686

Columns 40 to 49 
1.00000e-02 *
 -0.0477  0.0482  0.2298  1.6505  1.8471 -0.4714  3.6739 -3.5907 -2.2907  2.4591
[torch.FloatTensor of size 1x50]

In [323]:
x = [e(x_cat[:,i]) if (x_cat[:,i].data[0] > 0) else e(Variable(torch.LongTensor([0]))) for i, e in enumerate(embs)]

In [325]:
len(x)

11

In [326]:
# Need to handle the -1 in the data cleaning

In [293]:
xi = [x_cat[:,i] for i, e in enumerate(embs)][10]

In [297]:
for i, e in enumerate(embs):
    print(i, e)
    print(x_cat[:,i])
    e(x_cat[:,i])

0 Embedding(771770, 50)
Variable containing:
 6.7585e+05
[torch.LongTensor of size 1]

1 Embedding(29, 15)
Variable containing:
 19
[torch.LongTensor of size 1]

2 Embedding(1734, 50)
Variable containing:
 460
[torch.LongTensor of size 1]

3 Embedding(10, 5)
Variable containing:
 4
[torch.LongTensor of size 1]

4 Embedding(48, 24)
Variable containing:
 42
[torch.LongTensor of size 1]

5 Embedding(372, 50)
Variable containing:
 248
[torch.LongTensor of size 1]

6 Embedding(272, 50)
Variable containing:
-1
[torch.LongTensor of size 1]



RuntimeError: index out of range at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/TH/generic/THTensorMath.c:277

In [294]:
e(xi)

Variable containing:
 0.9752  1.3907
[torch.FloatTensor of size 1x2]

In [10]:
train, valid = train_test_split(TRN_CSV_COPY, train_size=1000, test_size=4)
sample = train.copy()

In [11]:
len(sample)

1000

In [12]:
sample.sample()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
1017223,20014ec274f3,495007,9,723,4,29,125,141,-1,668009,32146,1700.0,32.0,2017-03-25,1,2182a6861e9791abed31d3923d32d2aca0c3800739d072...,628.0,0.27307


In [13]:
sample = sample.drop(['image', 'activation_date'], axis=1)

In [33]:
class AvitoDataset(Dataset):
    def __init__(self, avito_df):
        self.avito_df = avito_df
        
        df_cat = self.avito_df[cat_vars]
        cat_cols = [c.values for n, c in df_cat.items()]
        self.cats = np.stack(cat_cols, 1).astype(np.int64)
        
        df_cont = self.avito_df[cont_vars]
        cont_cols = [c.values for n, c in df_cont.items()]
        self.cont = np.stack(cont_cols, 1).astype(np.float32)
        
        y = sample['deal_probability'].values
        self.y = np.stack([y], 1).astype(np.float32)
    
    def __len__(self):
        return len(self.avito_df)
    
    def __getitem__(self, idx):
        return [self.cats[idx], self.cont[idx], self.y[idx]]

In [34]:
train_df = AvitoDataset(sample)

In [35]:
train_df[0]

[array([459613,     14,   1107,      0,     41,     31,     -1,     -1, 104796, 675197,      0]),
 array([ 10500.,     90.,   2918.], dtype=float32),
 array([ 0.], dtype=float32)]

In [36]:
train_dl = DataLoader(train_df, batch_size=64, shuffle=True, num_workers=4)

In [37]:
df_cat = sample[cat_vars]

In [38]:
list(iter(train_dl))

[[
  
  Columns 0 to 5 
   5.7689e+05  4.0000e+00  3.1700e+02  6.0000e+00  0.0000e+00  2.7700e+02
   6.2991e+05  1.6000e+01  1.2760e+03  5.0000e+00  1.1000e+01  2.5700e+02
   7.5701e+05  1.7000e+01  1.5070e+03  6.0000e+00  0.0000e+00  2.7700e+02
   7.3135e+05  2.0000e+00  1.3600e+02  4.0000e+00  1.0000e+01  1.0400e+02
   4.6914e+05  3.0000e+00  6.7200e+02  0.0000e+00  4.1000e+01  3.1000e+01
   7.2303e+05  7.0000e+00  5.9200e+02  1.0000e+00  2.8000e+01  2.5900e+02
   4.0215e+05  0.0000e+00  1.2300e+02  4.0000e+00  2.9000e+01  1.2500e+02
   6.7845e+05  0.0000e+00  1.2300e+02  8.0000e+00  4.0000e+01  1.3000e+02
   3.5091e+05  1.0000e+01  7.4300e+02  5.0000e+00  1.5000e+01  2.5700e+02
   5.5734e+05  1.0000e+00  1.5890e+03  4.0000e+00  1.0000e+01  1.0400e+02
   5.3625e+05  7.0000e+00  1.3620e+03  4.0000e+00  4.6000e+01  5.9000e+01
   1.8201e+05  1.7000e+01  1.3000e+03  4.0000e+00  1.0000e+01  1.0900e+02
   3.5805e+04  5.0000e+00  1.7800e+02  3.0000e+00  1.2000e+01  2.7800e+02
   4.3512e+05 

In [39]:
x_cat, x_cont, y = next(iter(train_dl))

In [40]:
len(x_cat)

64

In [44]:
hidden = 100

In [144]:
class AvitoModel(nn.Module):
    def __init__(self):
        super(AvitoModel, self).__init__()
        self.fc1 = nn.Linear(3, hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden, 1)
        
        
    def forward(self,x_cat,x_cont):
        out = self.fc1(x_cont)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [145]:
model = AvitoModel()

In [146]:
model(Variable(x_cat),Variable(x_cont))

Variable containing:
-4.2307e+02
-8.5284e+01
-2.9148e+02
-1.2381e+03
-1.5369e+03
-3.9826e+05
-5.8158e+01
-4.0486e+01
-2.7369e+02
-2.8302e+05
-3.6792e+02
-6.4784e+01
-5.2168e+01
-3.4061e+04
 1.3119e+01
-3.4417e+01
-7.3613e+01
-3.0175e+02
 1.3563e+00
-1.1335e+02
-3.0463e+02
-3.7185e+02
-3.6929e+02
-6.2356e+01
-9.4765e+01
-1.5719e+02
-3.6915e+02
-5.3197e+02
-6.0276e+02
-5.2886e+02
-9.7514e+01
-9.2987e+02
-4.2255e+01
 5.0643e+02
-5.2620e+01
-2.0398e+04
 6.0715e+01
-9.4565e+01
-3.9671e+02
 3.3757e+00
[torch.FloatTensor of size 40x1]

In [147]:
v_x_cont = Variable(x_cont)

In [148]:
v_x_cont.shape

torch.Size([40, 3])

In [149]:
model

AvitoModel(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

In [150]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [164]:
num_epochs = 1000

In [165]:
for epoch in range(num_epochs):
    for i, data in enumerate(train_dl):
        x_cat, x_cont, y_label = data
        optimizer.zero_grad()
        outputs = model(Variable(x_cat), Variable(x_cont))
        loss = criterion(outputs, Variable(y_label))
        loss.backward()
        optimizer.step()
        print(loss.data[0])

3858.829833984375
2507.12109375
241.40325927734375
606.5445556640625
809.66259765625
591.468017578125
1855.579833984375
3868.324951171875
4791.36279296875
3919.657958984375
875.52099609375
1043.445556640625
496.647216796875
23643.810546875
1510.4302978515625
32772.84765625
356.6120300292969
358.54278564453125
3780.713623046875
575.312255859375
32574.201171875
12530.29296875
427.7368469238281
7521.19921875
766.851806640625
717.9789428710938
1756.643798828125
1280.4345703125
920.7755737304688
1318.030029296875
2386.464111328125
6061.17578125
4336.0146484375
247.49183654785156
400.65496826171875
431.1432189941406
857.2139892578125
1516.9488525390625
4981.93115234375
9413.3681640625
8707.84765625
263.0007629394531
273.9194641113281
16469.322265625
479.2659606933594
1029.7493896484375
21438.1328125
293.4701232910156
4193.46923828125
270.1661071777344
331.2966613769531
17910.572265625
1337.7769775390625
21443.47265625
5153.29931640625
5284.888671875
382.3227233886719
413.77154541015625
729.3

16217.193359375
456.3932800292969
3821.5634765625
7774.85498046875
3061.908447265625
17404.841796875
3761.0087890625
3832.240234375
1189.3250732421875
3790.803466796875
417.08636474609375
2625.029541015625
1086.2611083984375
417.47406005859375
3845.178466796875
358.71868896484375
10079.484375
201.09988403320312
1144.629150390625
3829.690185546875
1516.3441162109375
19871.005859375
271.1684265136719
7337.0771484375
750.6465454101562
378.67059326171875
16525.515625
1086.2401123046875
741.3982543945312
1811.7481689453125
4186.62255859375
252.5038604736328
2173.74755859375
20135.0390625
584.6657104492188
336.4649353027344
4087.51806640625
727.5686645507812
5599.07763671875
1058.655517578125
981.109130859375
90.7556381225586
10832.2294921875
1067.362548828125
4501.63818359375
16404.73828125
864.3814697265625
675.1182250976562
4125.1787109375
1267.8302001953125
668.5301513671875
394.9008483886719
321.2060852050781
400.2132568359375
1241.14599609375
2508.051513671875
3906.705078125
19766.8847

762.1123046875
23369.330078125
1157.681884765625
256.6922302246094
1890.1368408203125
2234.87353515625
284.8919677734375
3731.46240234375
19711.19140625
1309.9207763671875
887.4945678710938
4095.23876953125
4263.37890625
312.0607604980469
4532.9658203125
333.15411376953125
5247.078125
3728.414794921875
1155.9578857421875
1042.5072021484375
398.7330627441406
256.96966552734375
4078.006103515625
3793.134765625
2298.081787109375
19635.4609375
399.5966491699219
7626.462890625
2387.56494140625
245.73727416992188
16485.611328125
316.70599365234375
1771.8724365234375
4783.9345703125
708.451416015625
16387.46875
329.2030029296875
666.234375
355.91986083984375
4237.8486328125
1008.9860229492188
4649.57373046875
271.58551025390625
17734.083984375
7494.8837890625
4368.61083984375
3954.75634765625
358.80694580078125
4829.62255859375
1995.7322998046875
3669.925537109375
785.5738525390625
1472.3134765625
1291.8299560546875
198.87232971191406
439.65240478515625
7286.50634765625
3853.202392578125
454.

1271.9295654296875
19534.359375
1733.7513427734375
14289.015625
3819.335205078125
160.30902099609375
11297.119140625
20028.3359375
4298.1015625
4464.7216796875
1820.1441650390625
365.7480163574219
1422.9129638671875
16168.958984375
1687.808837890625
228.05328369140625
741.14306640625
1002.7003173828125
228.64199829101562
169.26332092285156
3803.4541015625
604.0839233398438
339.3743896484375
239.33338928222656
7837.22021484375
2600.98974609375
536.6361083984375
16455.810546875
4320.896484375
997.3954467773438
467.3683166503906
23309.17578125
694.5538330078125
7329.85498046875
1811.8802490234375
684.0289916992188
292.71038818359375
253.11251831054688
7377.2666015625
16753.701171875
354.9775695800781
4173.9990234375
4568.51904296875
3496.13623046875
17245.48046875
472.0104064941406
4977.90380859375
992.7075805664062
1059.5296630859375
1921.7633056640625
3597.680908203125
212.83206176757812
694.5629272460938
228.3878173828125
8522.271484375
3982.353271484375
602.3128662109375
366.806335449

317.2540283203125
520.100341796875
635.5917358398438
3798.547119140625
565.6239624023438
3474.74365234375
11185.560546875
1749.6529541015625
16534.0859375
476.9171447753906
3769.529052734375
2397.045654296875
19661.96484375
1334.373046875
296.32720947265625
851.3292236328125
3933.90087890625
480.15966796875
16878.697265625
7281.32421875
369.7578125
4384.765625
3513.305908203125
1369.0772705078125
2227.61865234375
420.4627380371094
3490.5068359375
815.6950073242188
3600.3486328125
1743.2601318359375
16153.1982421875
886.4880981445312
274.5077209472656
698.3264770507812
506.5799255371094
298.909912109375
3555.4716796875
7295.3056640625
8484.17578125
17006.654296875
301.2616271972656
320.2361755371094
614.6765747070312
3735.6875
3433.52392578125
708.2318725585938
16151.5302734375
6093.35302734375
16115.18359375
294.34844970703125
649.8945922851562
334.90301513671875
278.6584167480469
22679.779296875
398.62591552734375
10632.0673828125
2817.241455078125
620.189208984375
591.6127319335938
2

...

2546.62109375
1245.85595703125
378.8349304199219
238.7913360595703
1249.15625
6559.71435546875
254.53599548339844
896.8513793945312
16809.455078125
299.7665100097656
307.23016357421875
288.4111022949219
2571.504638671875
183.15673828125
1048.6212158203125
4445.54833984375
6118.2080078125
579.4390869140625
2597.071533203125
672.8737182617188
251.33226013183594
14552.6181640625
621.5542602539062
16638.466796875
2090.48193359375
279.8409729003906
357.3373718261719
5992.49658203125
214.4337158203125
16963.916015625
2669.815673828125
1016.227783203125
317.96044921875
334.9399108886719
2579.671142578125
1775.2021484375
14625.9853515625
684.7749633789062
226.34799194335938
292.2043762207031
368.57012939453125
4876.49658203125
336.52008056640625
273.80181884765625
2433.62158203125
2459.3916015625
1163.281005859375
240.76028442382812
160.0968017578125
5670.8271484375
611.7691650390625
17276.421875
3353.94921875
1492.2171630859375
507.44219970703125
224.0554962158203
2530.706787109375
14603.1982

15361.4453125
318.8636169433594
228.4860076904297
373.5460510253906
14569.552734375
242.0529327392578
8100.130859375
311.6048278808594
1511.048583984375
300.2356262207031
3401.25390625
518.42626953125
6835.22265625
356.74798583984375
231.5723419189453
145.38815307617188
154.51161193847656
476.47076416015625
2557.96630859375
248.07839965820312
6522.67431640625
2369.841796875
298.6872863769531
957.0040283203125
142.0146942138672
273.2754211425781
1152.9786376953125
321.55230712890625
15640.24609375
239.0528106689453
16831.0390625
7279.58984375
684.5960693359375
6613.76123046875
367.4908447265625
1118.51953125
299.31524658203125
278.60205078125
2422.31103515625
590.1409912109375
14582.9931640625
470.01129150390625
16728.56640625
2546.837646484375
353.08489990234375
2351.544677734375
1018.3574829101562
3675.361328125
2612.8896484375
314.5400085449219
335.2417297363281
1112.4259033203125
1330.5914306640625
2400.7255859375
473.4554443359375
464.60174560546875
317.8756408691406
741.9852905273

3417.416015625
828.1790161132812
203.31417846679688
14492.640625
395.7718505859375
204.57273864746094
2526.99267578125
5927.7314453125
348.00457763671875
2338.781494140625
3448.4609375
575.810791015625
5619.95263671875
14429.322265625
15038.0283203125
465.1221923828125
249.02415466308594
309.7172546386719
340.30078125
2440.928466796875
2566.181884765625
2363.01904296875
1702.1387939453125
3530.598388671875
217.05137634277344
297.510986328125
3471.940673828125
2250.889892578125
347.65484619140625
2306.383544921875
971.5845947265625
211.60997009277344
15049.80078125
715.378173828125
7850.1484375
252.22817993164062
644.5962524414062
830.3683471679688
16679.06640625
930.3190307617188
426.99871826171875
484.44464111328125
3762.208251953125
321.9430847167969
785.7754516601562
712.9208374023438
2431.984375
237.62020874023438
18596.4609375
409.7729797363281
261.296142578125
16306.0693359375
2930.7666015625
5860.18408203125
307.1593933105469
2253.69384765625
259.7049560546875
512.5662231445312


2402.89599609375
256.08056640625
584.6172485351562
5774.578125
383.2487487792969
16637.2578125
14376.1875
222.43521118164062
277.6438903808594
4405.35986328125
244.6103973388672
1091.281494140625
5602.03125
155.54917907714844
2311.714111328125
1206.15771484375
2362.47705078125
307.42138671875
501.7508850097656
160.19566345214844
261.3336486816406
2278.32568359375
2182.429931640625
2735.780517578125
15857.96484375
14669.5625
349.54644775390625
14985.37890625
306.05047607421875
15499.9521484375
2207.9208984375
2446.67822265625
2336.71533203125
166.8714141845703
986.192626953125
794.150634765625
919.795166015625
2627.694580078125
215.06884765625
7677.28564453125
253.562744140625
227.86888122558594
382.15277099609375
443.7272033691406
231.9784698486328
354.48577880859375
20488.59375
879.103271484375
16948.5703125
573.1517944335938
917.5344848632812
2306.095458984375
323.5536804199219
3041.36962890625
2408.023193359375
2237.953369140625
213.4205780029297
390.91827392578125
186.1895446777343

420.7728576660156
194.39808654785156
2947.529541015625
244.21295166015625
1174.3660888671875
150.92527770996094
5585.2236328125
2596.58935546875
278.5916442871094
2214.41357421875
934.7205200195312
22900.525390625
852.4603271484375
771.7186279296875
1027.5137939453125
28461.24609375
7766.66015625
319.8380432128906
794.0787963867188
1071.292724609375
220.16098022460938
414.90692138671875
292.3215026855469
2368.498046875
230.8400421142578
473.62701416015625
2218.39208984375
6744.84521484375


In [166]:
model(Variable(x_cat), Variable(x_cont))

Variable containing:
  60.7113
  30.8909
  -0.0471
  -2.6581
   9.0605
   0.0174
   0.0174
   0.0174
   3.3637
-358.4079
  31.4164
   0.0174
  -0.8605
  -0.7206
 -22.7918
  -0.0471
  -3.3088
   0.0174
   0.0174
  18.3334
   0.0174
  55.5664
   7.8848
   0.0174
-358.4079
   0.0174
  48.2149
   1.7129
  -3.6263
   2.0792
  -6.7955
  -5.3616
  17.8922
  -0.5442
  -6.7840
  -0.6449
   0.1715
  -0.2602
   0.5449
  -0.1223
[torch.FloatTensor of size 40x1]

In [167]:
y_label


 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.1693
 0.1270
 0.0000
 0.0000
 0.1277
 0.0000
 0.0651
 0.0000
 0.1246
 0.8652
 0.0000
 0.0000
 0.0261
 0.0000
 0.0707
 0.0495
 0.0000
 0.0000
 0.0000
 0.2439
 0.0000
 0.0856
 0.0000
 0.0000
 0.0000
 0.1287
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
 0.0000
[torch.FloatTensor of size 40x1]

In [168]:
loss.item()

AttributeError: 'Variable' object has no attribute 'item'

In [ ]:
[]

In [171]:
Variable(x_cat)

Variable containing:

Columns 0 to 5 
 7.2938e+04  0.0000e+00  1.6800e+02  0.0000e+00  2.0000e+00  4.2000e+01
 4.3224e+05  0.0000e+00  1.6800e+02  8.0000e+00  4.0000e+01  2.3300e+02
 3.3522e+05  1.1000e+01  1.5650e+03  2.0000e+00  3.8000e+01  3.1000e+02
 1.7362e+05  2.2000e+01  1.5260e+03  0.0000e+00  2.0000e+00  2.1100e+02
 1.6026e+05  1.2000e+01  1.0480e+03  4.0000e+00  2.9000e+01  1.2500e+02
 3.8430e+05  1.8000e+01  1.2400e+03  5.0000e+00  1.5000e+01  2.5700e+02
 7.0918e+05  9.0000e+00  1.4120e+03  4.0000e+00  4.2000e+01  3.2100e+02
 2.5860e+05  2.0000e+01  1.3980e+03  4.0000e+00  1.0000e+01  1.0400e+02
 7.4127e+04  1.2000e+01  1.0480e+03  4.0000e+00  4.6000e+01  3.6700e+02
 2.8958e+04  6.0000e+00  2.3000e+02  5.0000e+00  1.5000e+01  1.7500e+02
 3.3421e+04  2.1000e+01  5.8500e+02  8.0000e+00  1.6000e+01  3.3900e+02
 4.9955e+04  1.1000e+01  2.1200e+02  5.0000e+00  1.3000e+01  2.5700e+02
 7.4403e+05  1.1000e+01  9.7500e+02  4.0000e+00  1.0000e+01  1.0400e+02
 6.0041e+05  1.9000e+01  4

In [172]:
cat_sz = [(c, len(sample[c].cat.categories)+1) for c in cat_vars]

AttributeError: Can only use .cat accessor with a 'category' dtype

In [177]:
[sample[c].cat.categories for c in cat_vars]

AttributeError: Can only use .cat accessor with a 'category' dtype